In [ ]:
from hyperopt import fmin, tpe, hp, Trials
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_squared_error

# Load dataset
df = pd.read_csv('preprocessed_data.csv')
X = df.drop('target_column', axis=1).values
y = df['target_column'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Reshape for LSTM model
X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Define objective function for Bayesian Optimization
def objective(params):
    model = build_lstm(input_shape=(X_train.shape[1], 1), num_layers=params['num_layers'], units=params['units'])
    
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train_reshaped, y_train, epochs=10, batch_size=32, verbose=0)
    
    y_pred = model.predict(X_test_reshaped)
    loss = mean_squared_error(y_test, y_pred)
    return loss

# Hyperparameter space
space = {
    'num_layers': hp.choice('num_layers', [2, 3, 4]),
    'units': hp.choice('units', [32, 64, 128])
}

# Set up trials and minimize objective
trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=20, trials=trials)

print(f"Best Hyperparameters: {best}")
